In [44]:
import matplotlib.pyplot as plt
import numpy as np
from skimage import filters, morphology
from aicsimageio import AICSImage

In [45]:
file_name = f"/Users/demir/Documents/Hva_AI/afstudeerproject/Images Carolina/20240312_CKR_Exp35_STHdhQ97HA_96h_+BafA_CCT1_635P_HA_580_A11_460L_2.lif"
# file_name = f"/Users/demir/Documents/Hva_AI/afstudeerproject/codes_yunus/8hr_BTZ_image1.tif" # van internet

img = AICSImage(file_name)
img.dims.order  # T, C, Z, big Y, big X, (S optional)
img.dask_data  # Dask chunks fall on tile boundaries, pull YX chunks out of the image


dask.array<transpose, shape=(1, 4, 8, 1024, 1024), dtype=uint16, chunksize=(1, 1, 8, 1024, 1024), chunktype=numpy.ndarray>

In [46]:

c2 = img.get_image_data("CZYX", T=0, C=3)  # Assuming channel 2 is C2

if c2.ndim == 4:
    c2 = np.squeeze(c2)  # otomatik olarak boyutu düşürür
    print("c2.shape:", c2.shape)



c2.shape: (8, 1024, 1024)


In [47]:
c2_2d = np.max(c2, axis=0)
print("c2_2d.shape:", c2_2d.shape)  # (1024, 1024) gibi olur


c2_2d.shape: (1024, 1024)


In [48]:
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models

# --- 1. Görüntüyü yükle (örneğin c2_2d) ---
image = c2_2d  # Eğer 2D görüntü kullanıyorsan, c2_2d olmalı

# --- 2. Cellpose modelini başlat ---
model = models.Cellpose(gpu=False, model_type='cyto')  # `gpu=True` eğer GPU varsa

# --- 3. Segmentasyon işlemi ---
masks, flows, styles, diams = model.eval(image, diameter=None, channels=[0,0], do_3D=False)

# --- 4. Sonuçları görselleştir ---
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

# Orijinal görüntü
ax[0].imshow(image, cmap='gray')
ax[0].set_title("Orijinal Görüntü")

# Cellpose ile segmentasyon sonucu
ax[1].imshow(masks, cmap='nipy_spectral')
ax[1].set_title("Cellpose Segmentasyonu")

plt.tight_layout()
plt.show()

print(f"Bulunan hücre sayısı: {len(np.unique(masks)) - 1}")  # 0 boş arka planı saymamak için


In [ ]:
# Segmentasyonu görsel olarak kaydetmek
plt.imsave('cellpose_segmented_image.png', masks)


In [ ]:
# --- 4. Sonuçları görselleştir ---
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

# Orijinal görüntü
ax[0].imshow(image, cmap='gray')
ax[0].set_title("Orijinal Görüntü")

# Cellpose ile segmentasyon sonucu
ax[1].imshow(masks, cmap='nipy_spectral')
ax[1].set_title("Cellpose Segmentasyonu")

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models

# --- 1. Görüntüyü yükle (örneğin c2_2d veya c2_3d) ---
image = c2  # Eğer 3D görüntü kullanıyorsan, c2_3d olmalı

# --- 2. Cellpose modelini başlat ---
model = models.Cellpose(gpu=False, model_type='cyto')  # `gpu=True` eğer GPU varsa

# --- 3. Segmentasyon işlemi ---
masks, flows, styles, diams = model.eval(image, diameter=None, channels=[0,0], do_3D=True)

# --- 4. Sonuçları görselleştir ---
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

# Orijinal görüntü
ax[0].imshow(image[0], cmap='gray')  # 3D görüntüde bir Z-slice göster
ax[0].set_title("Orijinal Görüntü")

# Cellpose ile segmentasyon sonucu
ax[1].imshow(masks[0], cmap='nipy_spectral')  # 3D segmentasyonda bir Z-slice göster
ax[1].set_title("Cellpose Segmentasyonu")

plt.tight_layout()
plt.show()

print(f"Bulunan hücre sayısı: {len(np.unique(masks)) - 1}")  # 0 boş arka planı saymamak için
